# ABoffsets_LSSTComCam_c26202

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, and others

Created:  2024.11.15

## 1. Initial Setup...

In [1]:
%pwd

'/home/d/dltucker/WORK/GitHub/SP-1736/sciunit_photocalib/notebooks'

### 1.1 Import useful python packages

In [2]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

### 1.2 Include user input

In [3]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
repo = 'embargo'
#collections = 'LSSTComCam/runs/DRP/20241101_20241113/w_2024_46/DM-47566'
#collections = 'LSSTComCam/runs/DRP/20241101_20241120/w_2024_47/DM-47746'
collections = 'LSSTComCam/runs/DRP/20241101_20241204/w_2024_49/DM-47988'
instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'
day_obs_start = 20241101
day_obs_end = 20241204

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec C26202 spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '~/Downloads/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '~/Downloads/c26202_mod_008.fits'
               }

# RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [4]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. Calculate Synthetic AB magnitudes for C26202, based on official filter bandpasses

***Tina, we should check that I did not mess up things here!--December 13*** 

***In particular, are we getting the same synthetic mags for the various SEDs and filters as we got before?***

### 2.1 Change detectors from (default) LSST to ComCam

In [5]:
defaultDirs = st.setDefaultDirs()
defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)


### 2.2 Calculate synthetic mags

In [6]:
mags = {}

# Loop through all SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])
    
    

stiswfcnic_007 ~/Downloads/c26202_stiswfcnic_007.fits
mod_008 ~/Downloads/c26202_mod_008.fits


### 2.3 Convert mags numpy arrays into a pandas dataframe

In [7]:
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572793,17.586599
g,16.691920,16.694757
r,16.362021,16.361406
i,16.260196,16.258873
z,16.243678,16.243678
y,16.238847,16.239346


## 3. Query USDF Butler for ComCam measurements of C26202

### 3.1 Instantiate Butler

In [8]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `calexp`'s that overlap the sky position of C26202

#### 3.2.1 Find the `dataId`'s for all `calexp`'s in this repo/collection that overlap the RA, DEC of C26202

In [9]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)

print(f"\nFound {len(datasetRefs)} calexps")

0 {instrument: 'LSSTComCam', detector: 0, visit: 2024111000112, band: 'i', day_obs: 20241110, physical_filter: 'i_06'}
1 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100074, band: 'i', day_obs: 20241111, physical_filter: 'i_06'}
2 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100075, band: 'r', day_obs: 20241111, physical_filter: 'r_03'}
3 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100087, band: 'r', day_obs: 20241111, physical_filter: 'r_03'}
4 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100088, band: 'i', day_obs: 20241111, physical_filter: 'i_06'}
5 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100093, band: 'i', day_obs: 20241111, physical_filter: 'i_06'}
6 {instrument: 'LSSTComCam', detector: 0, visit: 2024111100094, band: 'r', day_obs: 20241111, physical_filter: 'r_03'}
7 {instrument: 'LSSTComCam', detector: 0, visit: 2024111500238, band: 'r', day_obs: 20241115, physical_filter: 'r_03'}
8 {instrument: 'LSSTComCam', detector: 0, visit:

#### 3.2.2 Plot the cutouts for all these `calexp`'s

**Tina, could use add some code here from the DP02_04b_Intermediate_Butler_Queries tutorial notebook so we can view these `calexp` images?  Maybe something from Section 3.1 from that tutorial notebook.  It would be good to take a look at the individual images in case there are any weird `calexp` images that we should ignore.**

#### 3.2.3 Create a pandas Dataframe containing the `sourceTable` info for all these `calexp`'s

Now, loop over the `datasetRefs` again, but this time grab the contents of the `sourceTable` table for each `ref` and combine into all into one big pandas DataFrame.  

In [10]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('sourceTable', dataId=dataId)
    src_list.append(src)
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


0 Visit 2024111000112, Detector 0:  Retrieved catalog of 1006 sources.
1 Visit 2024111100074, Detector 0:  Retrieved catalog of 1562 sources.
2 Visit 2024111100075, Detector 0:  Retrieved catalog of 1204 sources.
3 Visit 2024111100087, Detector 0:  Retrieved catalog of 1488 sources.
4 Visit 2024111100088, Detector 0:  Retrieved catalog of 1973 sources.
5 Visit 2024111100093, Detector 0:  Retrieved catalog of 1854 sources.
6 Visit 2024111100094, Detector 0:  Retrieved catalog of 1489 sources.
7 Visit 2024111500238, Detector 0:  Retrieved catalog of 1406 sources.
8 Visit 2024111500286, Detector 0:  Retrieved catalog of 1167 sources.
9 Visit 2024111500287, Detector 0:  Retrieved catalog of 1847 sources.
10 Visit 2024111600115, Detector 0:  Retrieved catalog of 1539 sources.
11 Visit 2024111700126, Detector 0:  Retrieved catalog of 2393 sources.
12 Visit 2024111700127, Detector 0:  Retrieved catalog of 2422 sources.
13 Visit 2024111700128, Detector 0:  Retrieved catalog of 2517 sources.
14

Let's look at the result:

In [11]:
src_all

,coord_ra,coord_dec,visit,detector,parentSourceId,x,y,xErr,yErr,ra,...,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,band,physical_filter
0,53.190168,-27.735114,2024111000112,0,0,736.000000,17.000000,NaN,NaN,53.190168,...,True,False,False,False,False,False,False,True,i,i_06
1,53.221834,-27.809401,2024111000112,0,0,2162.000000,15.000000,NaN,NaN,53.221834,...,True,False,False,False,False,False,False,True,i,i_06
2,53.239591,-27.851079,2024111000112,0,0,2962.211673,14.385582,0.255396,0.496841,53.239591,...,True,False,False,False,False,False,False,True,i,i_06
3,53.262796,-27.908386,2024111000112,0,0,4056.000000,32.000000,NaN,NaN,53.262796,...,True,False,False,False,False,False,False,True,i,i_06
4,53.200978,-27.765021,2024111000112,0,0,1299.147713,44.899586,1.583083,1.048285,53.200978,...,False,True,False,False,False,False,False,False,i,i_06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358219,53.305424,-27.919816,2024120300170,8,191755010453801261,2572.301756,3985.305110,0.725522,0.453763,53.305424,...,True,True,False,False,False,False,False,True,r,r_03
1358220,53.322323,-28.030237,2024120300170,8,191755010453801262,570.876013,3965.232369,0.155732,0.100224,53.322323,...,False,False,False,False,False,False,False,True,r,r_03
1358221,53.323749,-28.030771,2024120300170,8,191755010453801262,558.108369,3986.221634,0.397867,0.383604,53.323749,...,True,False,False,False,False,False,False,True,r,r_03
1358222,53.326865,-28.048204,2024120300170,8,191755010453801267,241.000000,3990.000000,NaN,NaN,53.326865,...,True,False,False,False,False,False,False,True,r,r_03


#### 3.2.4 Save `src_all` as a CSV file

Let's save `src_all` as a CSV file that we can download and examine with TOPCAT:

***(Rename this file to something else???)***

In [12]:
src_all.to_csv('LSSTComCam_C26202_fields.csv', index=False)

### 3.3 Extract only those rows containing C26202 from the src_all catalog

In [13]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
        (~src_all.extendedness_flag) & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# Create SkyCoord object for the coordinates of C26202
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=src_all_cleaned['ra'].values*u.degree, 
                     dec=src_all_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = src_all_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = src_all_cleaned[mask_sep].copy()
nearby_good_df['separation_c26202'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_c26202').drop_duplicates(subset=orig_columns, keep='first')


Add magCalib and magCalibErr columns:

In [14]:
# Flux in nano-Janskys to AB magnitudes:
best_df['magCalib'] = -2.5*np.log10(best_df['calibFlux']) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['magCalibErr'] = 2.5/math.log(10)*best_df['calibFluxErr']/best_df['calibFlux']

Display `visit`, `detector`, `band`, `calibFlux`, `calibFluxErr`, `magCalib`, `magCalibErr`, and `separation_c26202` from best_df, sorted by `visit` and `band`:

In [15]:
# Set pandas to show all rows...
pd.set_option("display.max_rows", None)

In [16]:
best_df[['visit', 'detector', 'band', 'calibFlux', 'calibFluxErr', 'magCalib', 'magCalibErr', 'separation_c26202']].sort_values(['visit', 'band'])

,visit,detector,band,calibFlux,calibFluxErr,magCalib,magCalibErr,separation_c26202
892115,2024110800247,5,r,1.121710e+06,1086.944206,16.275298,0.001052,0.605933
897244,2024110800248,5,i,1.206213e+06,1349.758913,16.196440,0.001215,0.599646
897737,2024110800249,5,i,1.184525e+06,1285.945871,16.216140,0.001179,0.627821
903263,2024110800250,5,r,1.120738e+06,1058.654941,16.276240,0.001026,0.625697
905931,2024110800251,5,r,1.129333e+06,1128.234145,16.267945,0.001085,0.613811
910241,2024110800252,5,i,1.210019e+06,1386.265216,16.193020,0.001244,0.614760
913296,2024110800255,5,r,1.132290e+06,1050.153348,16.265106,0.001007,0.615807
915044,2024110800256,5,i,1.204401e+06,1289.534959,16.198072,0.001162,0.612957
926581,2024110800259,5,r,1.136116e+06,1116.312916,16.261444,0.001067,0.604713
929773,2024110800260,5,i,1.219325e+06,1361.475855,16.184701,0.001212,0.600235


In [17]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

Number of rows:  396


In [18]:
# Reset pandas to its default maximum rows to print to screen
pd.reset_option("display.max_rows")

***Do we need to do any further masking/culling in the above table before proceeding?***

***Save to CSV file???***

***(How to name this file???)***

### 3.4 Match pandas dataframe with observed ComCam magnitudes (best_df) with pandas dataframe with the synthetic magnitudes (df_mags)

***TO DO!!!***

***Save results to CSV file???***

***(How to name this file???)***

**Let's stop here for now:**

In [19]:
raise StopExecution

## 4. Measure differences between the Observed ComCam and the LSST Synthetic Mags for C26202

***Calculate statistics from the matched dataframe from Section 3 above.***

In [ ]:
# DES DR2 AB offsets based on c26202_stisnic_007.fits from William Wester's DES-doc#15451...

print "AB offsets based on c26202_stisnic_007.fits"
print "==========================================="
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_wavg-r_wavg) - (g_ww-r_ww)
aboffset_ri = (r_wavg-i_wavg) - (r_ww-i_ww)
aboffset_iz = (i_wavg-z_wavg) - (i_ww-z_ww)
aboffset_zY = (z_wavg-Y_wavg) - (z_ww-Y_ww)

print "WAVG offsets"
print "------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_auto-r_auto) - (g_ww-r_ww)
aboffset_ri = (r_auto-i_auto) - (r_ww-i_ww)
aboffset_iz = (i_auto-z_auto) - (i_ww-z_ww)
aboffset_zY = (z_auto-Y_auto) - (z_ww-Y_ww)

print "MAG_AUTO offsets"
print "----------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 


## 5. Sandbox

In [ ]:
datasetRefs = butler.query_datasets("visitSummary", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

#print(datasetRefs)

print(f"\nFound {len(datasetRefs)} calexps")

In [ ]:
# Retrieve sourceTable for this visit & detector...

datasetType = 'sourceTable'
#dataId = {'visit': visit, 'detector': detector}
dataId = ref.dataId['visit']
dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
print(dataId)

src = butler.get(datasetType, dataId=dataId)

print(f"Retrieved catalog of {len(src)} sources.")

In [ ]:
butler.registry.queryDataIds(dimensions=('exposure'))

In [ ]:
icExp = butler.get('icExp', dataId=dataId)

In [ ]:
icExp_info = icExp.getInfo()

In [ ]:
print(icExp_info.getMetadata())

In [ ]:
print(dataId)

In [ ]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*src*')):
    print(dt)

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*icSrc*')):
    print(dt)

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)

In [ ]:
icSrc.asAstropy()

In [ ]:
src1 = butler.get('src', dataId=dataId)

In [ ]:
src1.asAstropy()